# Imports

In [2]:
import pandas as pd

In [3]:
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, load_metric
from sklearn.model_selection import train_test_split
import pandas as pd
from torch.utils.data import Dataset

/opt/anaconda3/envs/wsi-genai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#pip install --upgrade accelerate

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
import torch
print(torch.__version__)

2.6.0


In [7]:
torch.version.cuda

In [26]:
if torch.cuda.is_available():
    print("CUDA is available. Training on GPU.")
    device = torch.device("cuda")

elif  torch.backends.mps.is_available():
    device = torch.device("mps") 
else:
    print("CUDA is not available. Training on CPU.")
    device = torch.device("cpu")


print(device)


mps


In [27]:
data = pd.read_feather("../data/movie_reviews_4k.feather")

In [28]:
data.shape

(4000, 2)

In [29]:
data

,text,label
0,I wanted to vote zero or lower. I loved the co...,0
1,"Karen(Bobbie Phillips)mentions, after one of h...",0
2,This review applies for the cut of the film th...,0
3,"The best film on the battle of San Antonio, Te...",1
4,"In theory, 'Director's Commentary' should have...",0
...,...,...
3995,Excellent show. Instead of watching the same o...,1
3996,"It's hard to believe an ""action"" packed Jet Li...",0
3997,Me and my girlfriend went to see this movie as...,0
3998,This movie is my all time favorite!!! You real...,1


## Load Tokenizer

In [30]:
class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [31]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

## Train test split

In [32]:
texts = data['text'].tolist()
labels = data['label'].tolist()

encodings = tokenizer(texts, truncation=True, padding=True, max_length=128)

df_encodings = pd.DataFrame({'input_ids': encodings['input_ids'], 
                             'attention_mask': encodings['attention_mask'], 
                             'labels': labels})

In [33]:
train_df, val_df = train_test_split(df_encodings, test_size=0.2, random_state=42)

In [34]:
train_dataset = SentimentDataset({'input_ids': train_df['input_ids'].tolist(), 
                                  'attention_mask': train_df['attention_mask'].tolist()}, 
                                 train_df['labels'].tolist())

val_dataset = SentimentDataset({'input_ids': val_df['input_ids'].tolist(), 
                                'attention_mask': val_df['attention_mask'].tolist()}, 
                               val_df['labels'].tolist())

## Load Pretrained model

In [35]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Setup training Hyperparams

In [36]:
training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=3,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=64,   
    warmup_steps=50,                
    weight_decay=0.01,               
    logging_dir='./logs',            
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

## Train and evaluate

In [37]:
%%time
trainer.train()

Step,Training Loss
10,0.707100
20,0.690100
30,0.677000
40,0.629700
50,0.561200
60,0.568400
70,0.517400
80,0.460200
90,0.548500
100,0.441900


CPU times: user 30.6 s, sys: 10.8 s, total: 41.3 s
Wall time: 2min 13s


TrainOutput(global_step=600, training_loss=0.2996540061632792, metrics={'train_runtime': 133.6191, 'train_samples_per_second': 71.846, 'train_steps_per_second': 4.49, 'total_flos': 317921756774400.0, 'train_loss': 0.2996540061632792, 'epoch': 3.0})

In [38]:
trainer.evaluate()

{'eval_loss': 0.5503900051116943,
 'eval_runtime': 3.0446,
 'eval_samples_per_second': 262.764,
 'eval_steps_per_second': 4.27,
 'epoch': 3.0}

In [39]:
model.save_pretrained("sentiment_classification_DistillBert")

## Make classifications

In [40]:
def classify_sentences(model, tokenizer, sentences, device):
    # Tokenize the input
    encoded_input = tokenizer(
        sentences,
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=128
    )
    model.eval() 
    # Move inputs to the same device as the model
    inputs = {k: v.to(device) for k, v in encoded_input.items()}
    
    # (Optional) Ensure model is on the correct device as well
    # model.to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        
    # Move predictions back to CPU if you want to convert them to NumPy
    if device!='cpu':
        return torch.argmax(predictions, dim=-1).cpu().numpy()
    else:
        torch.argmax(predictions, dim=-1).numpy




In [42]:
model.eval() 

pred_sentences  =[ "I absolutely hate this movie, total dissaster", 
                  "Most beaytiful movie ever, I watched it 10 times, very good",
                  "Reasonably good movie"]

y_pred = classify_sentences(model, tokenizer, pred_sentences, device )

In [43]:
y_pred

array([0, 1, 1])

In [51]:
import numpy as np

In [58]:
y_pred.shape

(800,)

In [53]:
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)
val_data.reset_index(inplace=True, drop=True)

In [54]:
y_pred = classify_sentences(model, tokenizer, val_data.text.tolist(), device)

In [55]:
val_data["label_pred"] = y_pred

In [56]:
val_data["correct_prediction"] = val_data['label'] == val_data["label_pred"]

In [57]:
val_data["correct_prediction"].mean()

np.float64(0.8425)